In [1]:
! pip install selenium
! pip install beautifulsoup4

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time

In [48]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections
import pickle

In [3]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_path = "./chromedriver"

In [4]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [5]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)

In [6]:
comment_info_xpath = "//div[@id='main']"
show_more_button_xpath = "//span[@class='more-button style-scope ytd-comment-renderer']"
number_of_comments = "//h2[@id='count']/yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"
# comment_total_amount_xpath = "//yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"

In [7]:
df = pd.read_excel('DataSource.xlsx')
# remove the "who" and sort the table with "comedian"
df = df.drop('Who?', axis=1)
df = df.sort_values('Comedian')
df.head()

,Comedian,Title,Link of the transcript,Transcript,Video link
19,Amy Schumer,Porn Endings,https://scrapsfromtheloft.com/2017/09/08/amy-s...,"Well, I like to watch porn. Like, what girls i...",https://www.youtube.com/watch?v=EmzPp33kl7o
28,Arsenio Hall,Smart and Classy (2019),https://scrapsfromtheloft.com/2019/10/30/arsen...,Everything’s changing. This has changed a lot....,https://www.youtube.com/watch?v=c_KmV-hquvc&ab...
16,Aziz Ansari,So Sorry You Had a Baby,https://scrapsfromtheloft.com/2017/12/07/aziz-...,"I, uh, turned 30 years old this year. Yes, it’...",https://www.youtube.com/watch?v=lWA-xsAVzgE
38,CHRIS ROCK,TAMBORINE (2018),https://scrapsfromtheloft.com/2018/02/15/chris...,"Oh, man, the older you get, the more shit you ...",https://www.youtube.com/watch?v=dUzaSm0aAKw&ab...
37,CHRIS ROCK,TAMBORINE (2018),https://scrapsfromtheloft.com/2018/02/15/chris...,"Nah, man. But I love religion. I love watching...",https://www.youtube.com/watch?v=5bbKnj1-mR0&ab...


In [10]:
all_urls = df["Video link"]

In [11]:
web_list = []
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:    
    for url in all_urls:
        all_comments_info = []        
        prev_offest = -1
        offset = 0
        
        print(f"\nUrl: {url}")
        driver.get(url)
        # Scroll until the number of comments shows up
        while driver.wait_elements_xpath(number_of_comments, timeout=1, verbose=False) == None:
            driver.execute_script("scrollBy(0, 50)")
            
        # Scarpe comments until all the comments are collected
        while offset != prev_offest:
            prev_offest = offset
            driver.execute_script("scrollBy(0, 1000)")
            time.sleep(2)
            offset = driver.execute_script("return window.pageYOffset;");
            all_comments_info = driver.wait_elements_xpath(comment_info_xpath)
            print("", end=".")
        # Get whole html for us to find sth later
        html = driver.page_source    
        whole_page = bs(html,"html.parser")
        web_list.append(whole_page)


Url: https://www.youtube.com/watch?v=EmzPp33kl7o
......................................................
Url: https://www.youtube.com/watch?v=c_KmV-hquvc&ab_channel=NetflixIsAJoke
........
Url: https://www.youtube.com/watch?v=lWA-xsAVzgE
.....
Url: https://www.youtube.com/watch?v=dUzaSm0aAKw&ab_channel=NetflixIsAJoke
.....................................................................................
Url: https://www.youtube.com/watch?v=5bbKnj1-mR0&ab_channel=NetflixIsAJoke
.............
Url: https://www.youtube.com/watch?v=RIJavth0a9c
....
Url: https://www.youtube.com/watch?v=jxUQATD6Ezs&ab_channel=NetflixIsAJoke
.........................................................................................................................................................................................
Url: https://www.youtube.com/watch?v=30cYjy7_DDg&ab_channel=NetflixIsAJoke
............
Url: https://www.youtube.com/watch?v=nNx5tnqD9e0&ab_channel=JustForLaughs
.............................

In [12]:
len(web_list)

40

In [13]:
def html_to_list(web):
    whole_page = web
    comments = whole_page.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
    comments_names_list = []
    comments_contents_list = []
    comments_thumbsup_list = []

    for comment in comments:
        # reviewer's name
        name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()
        comments_names_list.append(name.strip('\n '))
        # reviewer's comment
        content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
        comments_contents_list.append(content)
        # reviewer's thumbsup amount
        thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
        new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
        comments_thumbsup_list.append(int(eval(new_thumbsup)))

    # list to df
    row = [comments_thumbsup_list, comments_names_list, comments_contents_list]
    df = pd.DataFrame(row).transpose()
    df.columns = ['Thumbsup', 'Name','Comment']
    return df

In [14]:
def get_all_user_names(web_list):
    name = {}
    for web in web_list:
        df = html_to_list(web)
        for n in df.Name:
            if n in name:
                name[n] += 1
            else:
                name[n] = 1
    return name

In [47]:
names = get_all_user_names(web_list)

In [52]:
names_df = pd.DataFrame(names.values(), index=names.keys(), columns=['number'])
names_df.sort_values(by='number', ascending=False).head(100)

,number
whitepride1004,31
rudeboymon,7
karissa and sweettweet,6
Audrey New,6
M A,5
...,...
R. A.,3
Anna markandan,3
Jeannie Russo,3
Xfirex Xbites,3


In [54]:
names_df.to_pickle('./pickles/web_list_first_40.plk')